In [1]:
# !pip install --force-reinstall -U -r requirements.txt --quiet

In [3]:
%%writefile agent3_doc_entity_extraction.py

from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
################
import boto3, json
import time
import os
from logging import exception
from botocore.exceptions import ClientError
# import trp
# from trp import Document
from typing import Dict

# Import date class from datetime module
# from datetime import date
# from datetime import datetime
import datetime
import decimal

from typing import Optional
import traceback
import botocore
from botocore.config import Config
##################################

# textract = boto3.client('textract', region_name='us-east-1')
# print("textract obj = ", textract)
print("in agent 3 beginning...")

app = BedrockAgentCoreApp()

# Initialize DynamoDB resource
dynamodb_resource = boto3.resource("dynamodb")

def get_extracted_entities(docid, classification, config_s3_path):
    try:
        print("classification = ", classification)
        # print("Event = ", event)
        # data_string = event['Records'][0]['body']   # this change is done only to accept the 'body' from SQS
        # print("event[body]",data_string,type(data_string))
        
        # if type(data_string) is dict:
        #     qtext = data_string
        # else:
        #     qtext = json.loads(data_string)
        # #print("\n","Lambda Handler context:",type(context),context)
        
        # print("qtext=json.loads(data_string) :",type(qtext),qtext)
        # s3files = [qtext['s3filename']]  # send the file name in an array
        # indexid = qtext['indexid']
        # print("indexid = ",indexid)
        # docid = qtext['docid']
        # print("docid = ",docid)


        # # Extract document ID from event
        # # docid = event.get('docid')
        # if not docid:
        #     print("Error: docid is not present")
        #     return {
        #         'statusCode': 400,
        #         'body': json.dumps({'error': 'docid is required'})
        #     }
        
        # Initialize Bedrock client
        bedrock = boto3.client('bedrock-runtime', region_name='us-west-2')
        
        # Get document extract details
        docs_extract_details = get_docs_extract(docid)        
        
        if 'Item' not in docs_extract_details:
            return {
                'statusCode': 404,
                'body': json.dumps({'error': 'Document not found for docid'})
            }
        
        # Extract document data
        document_name = docs_extract_details["Item"]["document_name"]
        print("document_name = ", document_name)
        rawtext = docs_extract_details["Item"]["rawtext"]
        tbltxt = docs_extract_details["Item"]["tbltxt"]
        keyvaluesText = docs_extract_details["Item"]["keyvaluesText"]
        # classification = docs_extract_details["Item"]["classification"]
        # print("classification = ", classification)
        prompt=""
        if classification == 'Legal':
            # Get entities template based on classification
            # legal_entities_to_be_extracted = get_legal_entities_template(classification)
            legal_entities_to_be_extracted = get_legal_entities_config(classification, config_s3_path)
            print("legal_entities_to_be_extracted = ", legal_entities_to_be_extracted)

            # Generate prompt and extract entities
            prompt = get_legal_prompt_ready(classification, str(rawtext), str(tbltxt), str(keyvaluesText), legal_entities_to_be_extracted)
        # else:
        #     # Get entities template based on classification
        #     entities_to_be_extracted = get_entities_template(classification)
        #     print("entities_to_be_extracted = ", entities_to_be_extracted)
            
        #     if not entities_to_be_extracted:
        #         return {
        #             'statusCode': 400,
        #             'body': json.dumps({'error': f'Unidentified Classification - {classification}'})
        #         }
            
        #     # Generate prompt and extract entities
        #     prompt = get_prompt_ready(classification, str(rawtext), str(tbltxt), str(keyvaluesText), entities_to_be_extracted)
        
        
        print("prompt before sending to LLM = ", prompt)
        llm_extracted_json = execute_model_ext(prompt, bedrock)
        print("llm_extracted_json = ", llm_extracted_json)
        
        # Parse and analyze results
        # parsed_json = json.loads(llm_extracted_json, strict=False)
        try:
            parsed_json = json.loads(llm_extracted_json, strict=False)
        except json.JSONDecodeError as e:
            print(f"JSON parsing failed: {e}")
            print(f"Raw LLM response: {llm_extracted_json}")
            return {'statusCode': 500, 'body': json.dumps({'error': 'Invalid JSON from LLM'})}

        print("parsed_json = ", parsed_json)
        empty_keys = extract_empty_keys(parsed_json)
        total_keys, empty_keys_count, empty_keys_percentage = calculate_stats(parsed_json)
        empty_key_perc = str(round(empty_keys_percentage)) + "%"
        ##############################
        gw_claim_number = "Not Available"
        if "claim_details_section" in parsed_json:
            print("inside if")
            if "claim_administrator_claim_number" in parsed_json["claim_details_section"]:
                print("inside if 1")
                if "value" in parsed_json["claim_details_section"]["claim_administrator_claim_number"]:
                    gw_claim_number = parsed_json["claim_details_section"]["claim_administrator_claim_number"]["value"]
                    print("gw_claim_number = ", gw_claim_number)
        
        if "CMS1500_section" in parsed_json:
            print("inside CMS if")
            if "other_claim_id" in parsed_json["CMS1500_section"]:
                print("inside CMS if 1")
                if "value" in parsed_json["CMS1500_section"]["other_claim_id"]:
                    gw_claim_number = parsed_json["CMS1500_section"]["other_claim_id"]["value"]
                    print("gw_claim_number in CMS = ", gw_claim_number)
        ##################################
        print("before updating hcltech-doc-extraction -> llm_extracted_json = ", llm_extracted_json)
        # Update Document Extraction table with entity results
        upsert_dashboard_record('hcltech-doc-extraction', docid=docid, 
                              extracted_entities=llm_extracted_json, 
                              total_keys=total_keys, 
                              empty_keys_count=empty_keys_count, 
                              empty_keys=empty_keys, 
                              empty_key_perc=empty_key_perc)
        print("before updating hcltech-dashboard ")
        # Update Dashboard table with entity extraction status
        upsert_dashboard_record('hcltech-dashboard', docid=docid, entity_extraction_status="Completed", gw_claim_id=gw_claim_number)
        #########################
        ########## From here we will call the confidence score lambda function 

        print("success to call entity confidence score lambda  ")
                                
        # try:
        #     # Initialize a boto3 client
        #     lambda_client = boto3.client('lambda')
        #     function_name = "hcltech_confidence_score_lambda"
        #     # Invoke the hcltech_entityexhcltech_confidence_score_lambdatraction_lambda function
        #     response = lambda_client.invoke(
        #         FunctionName=function_name,
        #         InvocationType='RequestResponse',  # Use 'Event' for asynchronous invocation
        #         Payload=json.dumps(event)  # Pass the event or any payload you need
        #     )
        #     print("lambda invoke response = ", response)
        # except Exception as e:
        #     print(f"Error invoking Entity Extraction Lambda function: {e}")


        ###############################
        return 'Entity extraction completed successfully and called confidencen score generator'
        # {
        #     'statusCode': 200,
        #     'body': json.dumps({
        #         'docid': docid,
        #         'classification': classification,
        #         'total_keys': total_keys,
        #         'empty_keys_count': empty_keys_count,
        #         'empty_key_percentage': empty_key_perc,
        #         'status': 'Entity extraction completed successfully'
        #     })
        # }
        
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }

def get_legal_prompt_ready(classification, raw_text, tabletext, key_value_pair_data, entities_to_be_extracted):
    """Generate prompt for entity extraction"""
    prompt = f"""Human: 
    
    You are an expert ai validation legal person of United States (both County and Federal) and an expert 
    document information extraction assistant specializing in parsing Worker Compensation {classification} documents. 
    Your task is to extract all key entities with precision.
    
    You are provided with '{classification}' document within the <raw_text>,  <key_value_pair_data> and <table_text> xml tags.
   
    <raw_text>{raw_text}</raw_text> , 
    <key_value_pair_data>{key_value_pair_data}</key_value_pair_data> , 
    <table_text>{tabletext}</table_text> 

    The raw text of '{classification}' document is within <raw_text> xml tag.
    The key value pair of '{classification}' document are mentioned as a list within <key_value_pair_data> xml tag.
    The tabular data of the '{classification}' document are mentioned as a json array with objects within <table_text> xml tag.     

    {entities_to_be_extracted} 

    Additional Guidelines:
    - Be extremely precise in extraction
    - If unsure about a value, still provide best possible guess
    - Confidence score should reflect extraction accuracy
    - Prioritize direct text matches over interpretations
    - Consider context and surrounding text for validation
    - Do not provide any supporting or explanation text beyond generating the perfect JSON.
    - Skip any preamble text and generate the final JSON ONLY. 
    - Generate only a perfect JSON till end.

    Provide the extracted information in the specified JSON format, ensuring comprehensive coverage of all document entities.

    
    Assistant:"""
    
    return prompt


def execute_model_ext(prompt, bedrock):
    """Execute Claude model for entity extraction"""
    try:
        print("inside execute_model() method. Please wait Large Language Model (Haiku) is preparing your JSON....")
        
        start_time = time.time()
        model_id = "anthropic.claude-3-haiku-20240307-v1:0"
        response = bedrock.invoke_model(
            modelId=model_id,
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 5000,
                "messages": [{
                    "role": "user",
                    "content": [{"type": "text", "text": prompt}],
                }]
            }),
        )
        
        end_time = time.time()
        print(f"Time taken by execute_model() {end_time - start_time} sec")
        
        result = json.loads(response.get("body").read())
        output_list = result["content"]
        for output in output_list:
            return output["text"]
            
    except Exception as error:
        print(f"Exception in execute_model(): {error}")
        print('Exception Details:', traceback.format_exc())
        return ""


def extract_empty_keys(data):
    """Extracts the keys from a JSON object whose values are empty or not filled."""
    empty_keys = []
    for key, value in data.items():
        if not value or value == "None":
            empty_keys.append(key)
        elif isinstance(value, dict):
            nested_empty_keys = extract_empty_keys(value)
            if nested_empty_keys:
                empty_keys.extend([f"{key}.{nested_key}" for nested_key in nested_empty_keys])
    return empty_keys

def calculate_stats(data):
    """Calculates the total number of keys and the percentage of keys with empty values."""
    total_keys = 0
    empty_keys = extract_empty_keys(data)
    empty_keys_count = len(empty_keys)

    def count_keys(d):
        nonlocal total_keys
        total_keys += len(d)
        for value in d.values():
            if isinstance(value, dict):
                count_keys(value)

    count_keys(data)
    empty_keys_percentage = (empty_keys_count / total_keys) * 100 if total_keys > 0 else 0

    return total_keys, empty_keys_count, empty_keys_percentage


def get_docs_extract(docid):
    """Retrieve document extract details from DynamoDB"""
    print('in get_docs_extract begin', docid)
    
    dynamodb_tbl_nm = "hcltech-doc-extraction"
    dbtable = dynamodb_resource.Table(dynamodb_tbl_nm)
    
    return dbtable.get_item(Key={'docid': docid})


def get_legal_entities_config(classification, config_s3_path):
    """Get entity extraction template based on document classification"""
    object_content = ""
    bucket_name = 'aimlusecases-pvt'  # Replace with your S3 bucket name
    object_key = config_s3_path # 'path/to/your/file.txt'  # Replace with the key (path) of your .txt file in S3
    s3_client = boto3.client('s3')
    print(f"@@@@@Contents of config_s3_path {config_s3_path}")
    if classification == 'Legal':
        try:
            response = s3_client.get_object(Bucket=bucket_name, Key=object_key)
            print("inside get_legal_entities_config() -> response = ", response)
            object_content = response['Body'].read().decode('utf-8')
            print(f"@@@@@@Contents of {object_key}: s3 config file after reading \n{object_content}")
        except Exception as e:
            print(f"Error reading object from S3: {e}")
   
    return object_content


def get_legal_entities_template(classification):
    """Get entity extraction template based on document classification"""
    if classification == 'Legal':
        return """
                Extraction Instructions:
                1. Carefully analyze the entire document text
                2. Extract entities exactly as they appear in the document
                3. For fields with no information, use "EMPTY"
                4. Provide DATE fields ONLY in MM/DD/YYYY format.

                Gather all the following information:
                1. What is the Case Number?
                2. What is the Name?
                3. Whether this document Category is 'Legal' or 'Non Legal'
                4. Whether this document is related to which 'Matter Type' from the following items ['Lawsuit', 'Arbitration','Hearng','Mediation']
                5. Whether this document is related to which 'Court Type' from the following items ['Federal','County']
                6. Whether this document is related to which 'Legal Speciality' from the following items ['Personal Injury','Motor Vehicle Liability', 'General Liability', 'Worker Compensation']
                7. What is the 'Primary Cause' in the document, pick from the following items ['Court Approval','Statute of Limitation', 'Valuation Dispute', 'Negotiation at Impasse', 'Unreasonable Demand', 'Blind Suit / First Notice', 'Low Settlement Offer', 'Predetermined', 'Delay or insufficient claimant']
                8. Whether this document sensitivity type is 'Sensitive' or 'Non Sensitive Document'
                9. What is the Claim Administration Claim Number?
                10. Provide the confidence for each entity.

                After gathering all the above information, respond in the form of following JSON along with confidence score for each entity:

                Required JSON Output Format:
                {
                "legal_section": {
                    "case_number": {
                    "value": "STRING",
                    "confidence": "FLOAT"
                    },
                    "name": {
                    "value": "STRING",
                    "confidence": "FLOAT"
                    },
                    "category": {
                    "value": "STRING",
                    "confidence": "FLOAT"
                    },
                    "matter_type": {
                    "value": "STRING",
                    "confidence": "FLOAT"
                    },
                    "court_type": {
                    "value": "STRING",
                    "confidence": "FLOAT"
                    },
                    "legal_speciality": {
                    "value": "STRING",
                    "confidence": "FLOAT"
                    },
                    "primary_cause": {
                    "value": "STRING",
                    "confidence": "FLOAT"
                    },
                    "sensitivity_type": {
                    "value": "STRING",
                    "confidence": "FLOAT"
                    },
                    "claim_administrator_claim_number": {
                    "value": "STRING",
                    "confidence": "FLOAT"
                    }
                }
                }

                CRITICAL: Return ONLY valid JSON. No explanations, no prefixes, no suffixes.
                Generate only a perfect JSON till end.
                Do not provide any supporting or explanation text beyond generating the perfect JSON.
                Skip any preamble text and generate the final JSON ONLY. 

                """
    return ""
    ###########################

def upsert_dashboard_record(tablename, docid, **kwargs):
    """
    Update existing record or insert new record in specified DynamoDB table

    Args:
        tablename (str): Name of the DynamoDB table
        docid (str): Document ID (primary key)
        **kwargs: Additional fields to update/insert

    Returns:
        dict: Response from DynamoDB operation
    """
    table = dynamodb_resource.Table(tablename)

    # Build update expression and attribute values
    update_expression = "SET "
    expression_attribute_values = {}

    for key, value in kwargs.items():
        update_expression += f"{key} = :{key}, "
        expression_attribute_values[f":{key}"] = value

    # Remove trailing comma and space
    update_expression = update_expression.rstrip(", ")
    print("update_expression =", update_expression)
    
    try:
        response = table.update_item(
            Key={'docid': docid},
            UpdateExpression=update_expression,
            ExpressionAttributeValues=expression_attribute_values,
            ReturnValues='ALL_NEW'
        )
        print("response =", response)
        return response
    except ClientError as e:
        print(f"Error updating record: {e}")
        raise


@tool
def extract_entities_based_on_config(docid, classification, s3files):
    """ Get doc_extraction """ # Dummy implementation
    print("inside doc_extraction if condition")
    print("inside doc_extraction receiving parameters = docid = ", docid, ", classification =  ", classification , ", s3files = ",s3files)
    try:
       
        print("inside get extract_entities_based_on_config if condition")
        # docid = event.get('docid')
        print("inside get extract_entities_based_on_config docid = ", docid)
        # s3files = event.get('s3files')
        print("inside get extract_entities_based_on_config s3files = ", s3files)
        # classification = event.get('classification')
        print("inside get extract_entities_based_on_config classification = ", classification)
        
        extracted_entities = get_extracted_entities(docid, classification, s3files)
        print("@@@@@@extracted_entities = ", extracted_entities)
        return {
            'statusCode': 200,
            'body': json.dumps({'extracted_entities': extracted_entities})
        }

        # return extracted_entities
        
    except Exception as e:
        print("Error Message : ", str(e))
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }

########################################

model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    # tools=[calculator, weather],
    # system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
    tools=[extract_entities_based_on_config],
    system_prompt="You're a helpful assistant. You can do entity extraction based on config file"
)

# # @app.entrypoint
# # def strands_agent_bedrock(payload):
# #     """
# #     Invoke the agent with a payload
# #     """
# #     user_input = payload.get("prompt")
# #     print("User input:", user_input)
# #     response = agent(user_input)
# #     return response.message['content'][0]['text']

# if __name__ == "__main__":
#     print("\@@@@@@@ agent3 doc entity extract beginning:")
#     user_input = "extract the entities of the document whose document id is DOC417927, classification is Legal and the s3 config file path is newmexicomutual/yaju/legal_entity_extraction_config.txt"
#     response = agent(user_input)
#     print("response = ", response)
#     # return response.message['content'][0]['text']
#     # app.run()

Overwriting agent3_doc_entity_extraction.py


In [6]:
!python agent3_doc_entity_extraction.py 

in agent 3 beginning...
\@@@@@@@ agent3 doc entity extract beginning:
I'll help you extract entities from the document with the provided details. I have all the required parameters to proceed with the entity extraction:
Tool #1: extract_entities_based_on_config
inside doc_extraction if condition
inside doc_extraction receiving parameters = docid =  DOC417927 , classification =   Legal , s3files =  newmexicomutual/yaju/legal_entity_extraction_config.txt
inside get extract_entities_based_on_config if condition
inside get extract_entities_based_on_config docid =  DOC417927
inside get extract_entities_based_on_config s3files =  newmexicomutual/yaju/legal_entity_extraction_config.txt
inside get extract_entities_based_on_config classification =  Legal
classification =  Legal
in get_docs_extract begin DOC417927
document_name =  newmexicomutual/claimforms/IN434221/DOC417927/LegalCaseDocument-WC.pdf
@@@@@Contents of config_s3_path newmexicomutual/yaju/legal_entity_extraction_config.txt
inside g